<a href="https://colab.research.google.com/github/marzione00/Mining_spark/blob/main/Mining_COVID19_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

base_path = "/content/drive/MyDrive/tweet_vaccines/"
import pandas as pd
tweets = pd.read_csv(base_path + "df_tweets_2021-07-15.csv", sep="|")
tweets.head(5)
#tweets=tweets.drop(range(5001, 15271))

Mounted at /content/drive


,hashtag,timestamp,tweet_text,user_name,language
0,#vaccines,2021-07-14 23:59:48,When I hear #Trudeau &amp; CO. self praise abo...,vitevoila,en
1,#vaccines,2021-07-14 23:58:48,Compliance to adjust oversight practices with ...,_UpFRONT_,en
2,#vaccines,2021-07-14 23:58:02,#COVID19 #vaccines risks/benefits:,bok_bok_ba_GOK,en
3,#vaccines,2021-07-14 23:55:57,Vietnam's health ministry has confirmed that U...,Madhuryavalli1,en
4,#vaccines,2021-07-14 23:53:32,"Why are so-called ""pro-choice liberals"" NOT pr...",PRAISETRIUNEGOD,en


In [29]:
import nltk
nltk.download('punkt')
tweets["tweet_text"] = tweets["tweet_text"].astype(str).apply(nltk.word_tokenize)
tweets.head(5)
#print (tweets.shape)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,hashtag,timestamp,tweet_text,user_name,language
0,#vaccines,2021-07-14 23:59:48,"[When, I, hear, #, Trudeau, &, amp, ;, CO., se...",vitevoila,en
1,#vaccines,2021-07-14 23:58:48,"[Compliance, to, adjust, oversight, practices,...",_UpFRONT_,en
2,#vaccines,2021-07-14 23:58:02,"[#, COVID19, #, vaccines, risks/benefits, :]",bok_bok_ba_GOK,en
3,#vaccines,2021-07-14 23:55:57,"[Vietnam, 's, health, ministry, has, confirmed...",Madhuryavalli1,en
4,#vaccines,2021-07-14 23:53:32,"[Why, are, so-called, ``, pro-choice, liberals...",PRAISETRIUNEGOD,en


In [32]:
#define stop words and punctuaction
import nltk, string
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_en = stopwords.words("english") 
stop_words_en = stop_words_en + list(string.punctuation)
add= ["''","...", "``", "--"]
stop_words_en.extend(add)
stop_words_en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [33]:
#clean the corpus
import nltk
nltk.download('wordnet')
import string
stop=set(stopwords.words('english'))
exclude=set(string.punctuation)
def clean(doc):
    stop_free=" ".join([i for i in doc.lower().split() if i not in stop])
    punc_free=''.join(ch for ch in stop_free if ch not in exclude)
    return punc_free

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
df = pd.DataFrame() 
lista = [] 
listb = []
for i in range (0,15271):
  for x in tweets.loc[i,"tweet_text"]:
     lista.append(i)
     listb.append(clean (x))
d = {'basket':lista,'item':listb}
dframe = pd.DataFrame(d)  
dframe.head(20)

,basket,item
0,0,
1,0,
2,0,hear
3,0,
4,0,trudeau
5,0,
6,0,amp
7,0,
8,0,co
9,0,self


In [39]:
import numpy as np
dframe['item'].replace('', np.nan, inplace=True)
dframe.head(20)

,basket,item
0,0,NaN
1,0,NaN
2,0,hear
3,0,NaN
4,0,trudeau
5,0,NaN
6,0,amp
7,0,NaN
8,0,co
9,0,self


In [41]:
dframe.dropna(subset=['item'], inplace=True)
dframe.head(50)

,basket,item
2,0,hear
4,0,trudeau
6,0,amp
8,0,co
9,0,self
10,0,praise
12,0,vaccine
13,0,supply
15,0,amp
17,0,progress


In [ ]:
!pip install pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [ ]:
df_sp = spark.createDataFrame(dframe)

In [ ]:
df_sp.createOrReplaceTempView("df_sp")

In [ ]:
df_sp.show(n = 30)

+------+--------+
|basket|    item|
+------+--------+
|     0|    When|
|     0|       I|
|     0|    hear|
|     0|       #|
|     0| Trudeau|
|     0|       &|
|     0|     amp|
|     0|       ;|
|     0|     CO.|
|     0|    self|
|     0|  praise|
|     0|   about|
|     0| vaccine|
|     0|  supply|
|     0|       &|
|     0|     amp|
|     0|       ;|
|     0|progress|
|     0|       ,|
|     0|       I|
|     0|     see|
|     0|      it|
|     0|      as|
|     0|       a|
|     0|   smack|
|     0|      in|
|     0|     the|
|     0|    face|
|     0|      to|
|     0|     our|
+------+--------+
only showing top 30 rows



In [ ]:
from pyspark.sql.functions import collect_set

baskets = df_sp.groupBy('basket').agg(collect_set('item').alias('items'))
baskets.createOrReplaceTempView('baskets')
baskets.toPandas().head(40)

,basket,items
0,26,"[the, ?, been, depressed, associated, that, wi..."
1,29,"[Vaccines, the, education, ,, No, @, and, unde..."
2,474,"[,, may, CovidCoalition, storage, health, ones..."
3,964,"[the, out, JohnsonVariant, website, on, UK, Jo..."
4,1677,"[Thailand, market, William67335893, @, in]"
5,1697,"[again, population, -, The, COVID, And, surge,..."
6,1806,"[,, serious, Take, health, 'surge, News, delta..."
7,1950,"[,, Mary, Senior, transmissible, Coronavirus, ..."
8,2040,"[,, the, is, ?, ASEAN, UNinMyanmar, UNHumanRig..."
9,2214,"[,, lives, can, vaccinated, faculty, including..."


In [ ]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0)
model = fpGrowth.fit(baskets)

In [ ]:
# Display frequent itemsets.
mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")
mostPopularItemInABasket.show()

# Display generated association rules.
associationRules = model.associationRules
associationRules.createOrReplaceTempView("associationRules")
associationRules.show()

# transform examines the input items against all the association rules and summarize the consequents as prediction
associations = model.transform(baskets)
associations.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|                 [#]|13926|
|                 [.]| 9150|
|              [., #]| 8785|
|               [the]| 7900|
|            [the, .]| 5735|
|         [the, ., #]| 5525|
|            [the, #]| 7522|
|                [to]| 7026|
|           [to, the]| 4473|
|        [to, the, .]| 3476|
|     [to, the, ., #]| 3342|
|        [to, the, #]| 4303|
|             [to, .]| 5062|
|          [to, ., #]| 4880|
|             [to, #]| 6681|
|      [DeltaVariant]| 6754|
| [DeltaVariant, the]| 3673|
|[DeltaVariant, th...| 2681|
|[DeltaVariant, th...| 2681|
|[DeltaVariant, th...| 3673|
+--------------------+-----+
only showing top 20 rows

+--------------------+----------+-------------------+------------------+-------------------+
|          antecedent|consequent|         confidence|              lift|            support|
+--------------------+----------+-------------------+------------------+-------------------+

In [ ]:
query = """select items, freq
           from mostPopularItemInABasket
           where size(items) > 2
           order by freq desc
           limit 20"""
spark.sql(query).show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|         [the, ., #]|5525|
|          [to, ., #]|4880|
|           [,, ., #]|4573|
|[DeltaVariant, ., #]|4358|
|        [to, the, #]|4303|
|         [,, the, #]|4004|
|           [@, ., #]|3889|
|          [of, ., #]|3880|
|        [of, the, #]|3751|
|    [vaccines, ., #]|3728|
|[DeltaVariant, th...|3673|
|          [,, to, #]|3545|
|        [to, the, .]|3476|
|         [and, ., #]|3394|
|     [to, the, ., #]|3342|
|  [vaccines, the, #]|3294|
|         [,, the, .]|3268|
|         [@, the, #]|3215|
|      [,, the, ., #]|3165|
|          [in, ., #]|3163|
+--------------------+----+



In [ ]:
query = """select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
           from associationRules
           order by confidence desc
           limit 20"""

spark.sql(query).show()

+--------------------+-----------------+----------+
|     antecedent (if)|consequent (then)|confidence|
+--------------------+-----------------+----------+
|[in, DeltaVariant...|              [#]|       1.0|
|[DeltaVariant, th...|              [#]|       1.0|
|              [&, #]|              [;]|       1.0|
|        [COVID19, .]|              [#]|       1.0|
|           [COVID19]|              [#]|       1.0|
|  [in, DeltaVariant]|              [#]|       1.0|
|         [amp, ;, #]|              [&]|       1.0|
|[of, DeltaVariant...|              [#]|       1.0|
|[COVID19, DeltaVa...|              [#]|       1.0|
|[of, DeltaVariant...|              [#]|       1.0|
|         [amp, &, #]|              [;]|       1.0|
|               [amp]|              [;]|       1.0|
|      [DeltaVariant]|              [#]|       1.0|
|   [DeltaVariant, .]|              [#]|       1.0|
|            [amp, ;]|              [&]|       1.0|
|            [amp, #]|              [;]|       1.0|
|  [of, Delt